## Memory 

There are several use cases where it is valuable to maintain a bank of useful facts that can be intelligently added to the context of the agent just before a specific step. The typically use case here is a RAG pattern where a query is used to retrieve relevant information from a database that is then added to the agent's context.


AgentChat provides a `Memory` protocol that can be extended to provide this functionality.  The key methods are `query`, `add`, `clear`, and `cleanup`. The `query` method is used to retrieve relevant information from the memory store, the `add` method is used to add new entries to the memory store, the `clear` method is used to clear all entries from the memory store, and the `cleanup` method is used to clean up any resources used by the memory store.


## ListMemory

ListMemory is a simple list-based memory implementation that uses text similarity matching to retrieve relevant information from the memory store. The similarity score is calculated using the `SequenceMatcher` class from the `difflib` module. The similarity score is calculated between the query text and the content text of each memory entry.   

In the following example, we will use ListMemory to similate a memory bank of user preferences and explore how it might be used in personalizing the agent's responses.

In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent 
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.memory._list_memory import ListMemory, MemoryContent, MemoryMimeType

# create a simple memory item 
user_memory = ListMemory()
await user_memory.add(MemoryContent(
    content="The weather should be in metric units",
    mime_type=MemoryMimeType.TEXT
))

await user_memory.add(MemoryContent(
    content="Meal recipe must be vegan",
    mime_type=MemoryMimeType.TEXT
))

async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 degrees and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 degrees and Sunny."  

assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06", 
    ),
    tools=[get_weather], 
    memory=[user_memory]
)
  
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition = TextMentionTermination("TERMINATE"))

# Run the team and stream messages to the console
stream = agent_team.run_stream(task="What is the weather in New York?")
await Console(stream)

---------- user ----------
What is the weather in New York?
---------- assistant_agent ----------
[FunctionCall(id='call_qNo7mjlNoVNaQzK1B6toXuW5', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
[Prompt tokens: 128, Completion tokens: 20]
---------- assistant_agent ----------
[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_qNo7mjlNoVNaQzK1B6toXuW5')]
---------- assistant_agent ----------
The weather in New York is 23 degrees and Sunny.
---------- assistant_agent ----------
The weather in New York is 23 degrees Celsius and sunny. TERMINATE
[Prompt tokens: 170, Completion tokens: 17]
---------- Summary ----------
Number of messages: 5
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 298
Total completion tokens: 37
Duration: 1.97 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=20), content=[FunctionCall(id='call_qNo7mjlNoVNaQzK1B6toXuW5', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_qNo7mjlNoVNaQzK1B6toXuW5')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant_agent', models_usage=None, content='The weather in New York is 23 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=17), content='The weather in New York is 23 degrees Celsius and sunny. TERMINATE', type='TextMes

We see above that the weather is returned in Centigrade as stated in the user preferences.

In [2]:
stream = agent_team.run_stream(task="Suggest a brief meal recipe")
await Console(stream)

---------- user ----------
Suggest a brief meal recipe
---------- assistant_agent ----------
Here's a brief vegan meal recipe for you:

**Vegan Chickpea Salad Sandwich**

**Ingredients:**
- 1 can chickpeas, drained and rinsed
- 2 tablespoons vegan mayonnaise
- 1 tablespoon Dijon mustard
- 1 tablespoon lemon juice
- Salt and pepper to taste
- 1/4 cup diced celery
- 1/4 cup diced red onion
- 2 tablespoons chopped fresh parsley
- 4 slices whole-grain bread
- Lettuce leaves and tomato slices (optional)

**Instructions:**
1. In a bowl, mash the chickpeas with a fork until mostly broken down, but still a bit chunky.
2. Stir in the vegan mayonnaise, Dijon mustard, lemon juice, salt, and pepper.
3. Add the diced celery, red onion, and chopped parsley. Mix until well combined.
4. Layer the chickpea salad on two slices of whole-grain bread. Add lettuce leaves and tomato slices if desired.
5. Place the remaining slices of bread on top to form sandwiches. Serve immediately and enjoy! 

This vegan 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Suggest a brief meal recipe', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=235, completion_tokens=239), content="Here's a brief vegan meal recipe for you:\n\n**Vegan Chickpea Salad Sandwich**\n\n**Ingredients:**\n- 1 can chickpeas, drained and rinsed\n- 2 tablespoons vegan mayonnaise\n- 1 tablespoon Dijon mustard\n- 1 tablespoon lemon juice\n- Salt and pepper to taste\n- 1/4 cup diced celery\n- 1/4 cup diced red onion\n- 2 tablespoons chopped fresh parsley\n- 4 slices whole-grain bread\n- Lettuce leaves and tomato slices (optional)\n\n**Instructions:**\n1. In a bowl, mash the chickpeas with a fork until mostly broken down, but still a bit chunky.\n2. Stir in the vegan mayonnaise, Dijon mustard, lemon juice, salt, and pepper.\n3. Add the diced celery, red onion, and chopped parsley. Mix until well combined.\n4. Layer the chickpea salad on two slices of whole-

## Vector DB Memory (ChromaDB)

Similarly, we can implement a memory store that uses a vector database to store and retrieve information.  `ChromaMemory` is a memory implementation that uses ChromaDB to store and retrieve information. ChromaDB is a vector database that is optimized for similarity search.  
 

In [2]:
# !pip install chromadb sentence-transformers

In [4]:

from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.memory._base_memory import MemoryContent, MemoryMimeType
from autogen_agentchat.memory._chroma_memory import ChromaMemory, ChromaMemoryConfig


# Initialize memory
chroma_memory = ChromaMemory(
    name="travel_memory",
    config=ChromaMemoryConfig(
        collection_name="travel_facts",
        k=1
    )
)

# Add travel-related memories
await chroma_memory.add(MemoryContent(

    content="Paris is known for the Eiffel Tower and amazing cuisine.",
    mime_type=MemoryMimeType.TEXT

))

await chroma_memory.add(MemoryContent( 
    content="The most important thing about tokyo is that it has the world's busiest railway station - Shinjuku Station.",
    mime_type=MemoryMimeType.TEXT

))
 

# Query needs ContentItem too
results = await chroma_memory.query(
    MemoryContent(
        content="Tell me about Tokyo.",
        mime_type=MemoryMimeType.TEXT
    )
)

print(len(results), results)

1 [MemoryQueryResult(content=MemoryContent(content="The most important thing about tokyo is that it has the world's busiest railway station - Shinjuku Station.", mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata={}, timestamp=datetime.datetime(2024, 12, 25, 7, 14, 36, 419091), source=None), score=0.5832697153091431)]


In [5]:
# Create agent with memory
agent = AssistantAgent(
    name="travel_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o",
        # api_key="your_api_key"
    ),
    memory=chroma_memory,
    system_message="You are a travel expert"
)

agent_team = RoundRobinGroupChat([agent], termination_condition = MaxMessageTermination(max_messages=2))
stream = agent_team.run_stream(task="Tell me the most important thing about Tokyo.")
await Console(stream);

---------- user ----------
Tell me the most important thing about Tokyo.
---------- travel_agent ----------
One of the most important aspects of Tokyo is its status as a major global hub for culture, technology, and commerce. A notable highlight is Shinjuku Station, which holds the title of the world's busiest railway station. This station is emblematic of Tokyo's extensive and efficient public transportation network that seamlessly connects millions of residents and tourists across the city and beyond. This sophisticated infrastructure is a reflection of Tokyo's dynamic urban environment, showcasing its blend of traditional and modern elements in architecture, culture, and lifestyle.
[Prompt tokens: 62, Completion tokens: 102]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 62
Total completion tokens: 102
Duration: 1.73 seconds
